In [1]:
import tensorflow as tf
import numpy as np
import datetime

In [2]:
(train_image,train_lbel),(test_image,test_label) = tf.keras.datasets.mnist.load_data()

In [3]:
train_image.shape

(60000, 28, 28)

In [4]:
train_image = np.expand_dims(train_image,axis=-1)
test_image = np.expand_dims(test_image,axis=-1)

In [5]:
train_image.shape,test_image.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
train_image = tf.cast(train_image/255,tf.float32)
test_image = tf.cast(test_image/255,tf.float32)

In [7]:
train_label = tf.cast(train_lbel,tf.int64)
test_label = tf.cast(test_label,tf.int64)

In [8]:
datasets = tf.data.Dataset.from_tensor_slices((train_image,train_lbel))
test_datasets = tf.data.Dataset.from_tensor_slices((test_image,test_label))

In [9]:
datasets

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.uint8)>

In [10]:
datasets = datasets.repeat().shuffle(60000).batch(128)

In [11]:
test_datasets = test_datasets.repeat().batch(128)

In [12]:
datasets

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.uint8)>

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(16,[3,3],activation='relu'),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(10,activation='softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 16)    160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [15]:
import os
log_dir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [16]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir,histogram_freq=1)

In [17]:
model.fit(datasets,
         epochs=5,
         steps_per_epoch=60000//128,
         validation_data=test_datasets,
         validation_steps=10000//128,
         callbacks=[tensorboard_callback])

Train for 468 steps, validate for 78 steps
Epoch 1/5
468/468 [==============================] - 7s 14ms/step - loss: 1.4904 - accuracy: 0.5440 - val_loss: 0.7700 - val_accuracy: 0.7608
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.6504 - accuracy: 0.7949 - val_loss: 0.5557 - val_accuracy: 0.8259
Epoch 3/5
468/468 [==============================] - 3s 6ms/step - loss: 0.5322 - accuracy: 0.8320 - val_loss: 0.4974 - val_accuracy: 0.8455
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.4849 - accuracy: 0.8477 - val_loss: 0.4492 - val_accuracy: 0.8558
Epoch 5/5
468/468 [==============================] - 3s 6ms/step - loss: 0.4504 - accuracy: 0.8576 - val_loss: 0.4305 - val_accuracy: 0.8613


In [18]:
%reload_ext tensorboard
%matplotlib inline

In [19]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 32044), started 1 day, 18:58:48 ago. (Use '!kill 32044' to kill it.)